In [1]:
from keras.layers import Input, Dense, GaussianNoise, Lambda, Dropout, concatenate, LSTM, Add, Multiply, Layer
from keras.models import Model, Sequential, load_model
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
from keras import backend as K
import numpy as np
import keras
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.utils import plot_model
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from utils import *
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Using TensorFlow backend.


In [2]:
H = K.constant([[1, 1]])
SNR1 = 7
SNR2 = 7
SNRs = [SNR1, SNR2]
ebno = [calc_ebno(SNR) for SNR in SNRs]

k = 2
n_channel = 2
M = 2 ** k
k = int(k)
R = k / n_channel

In [3]:
def TransmissionLayer(x, t, k):
    signal = H[t, k] * x

    for i in range(t):
        if i == k:
            continue
        interference = H[i, k] * x
        signal = signal + interference


#     noise = K.random_normal(K.shape(signal),
#                         mean=0,
#                         stddev=np.sqrt( 1/ (2 * R * ebno[k])))
    return signal 

In [4]:
class AlphaCallback(Callback):
    def __init__(self, alpha):
        super().__init__()
        self.alpha = alpha

    def on_batch_end(self, batch, logs=None):
        loss1, loss2 = logs["decoder1_loss"], logs["decoder2_loss"]
        K.set_value(self.alpha, loss1 / (loss1 + loss2))

In [5]:
input_signal1 = Input(shape=(M,), name="input1")
input_signal2 = Input(shape=(M,), name="input2")
encoder1 = Sequential([
    Dense(M, activation="relu"),
    Dense(n_channel, activation="linear")
], name="encoder1")
encoder2 = Sequential([
    Dense(M, activation="relu"),
    Dense(n_channel, activation="linear")
], name="encoder2")

combiner = Sequential([
    Dense(n_channel, activation="relu"),
    Dense(n_channel, activation="linear"),
    BatchNormalization(center=False, scale=False),
], name="combiner")

signal_input1 = Sequential(
    [Lambda(TransmissionLayer, arguments={"t":0, "k":0}),
    GaussianNoise(np.sqrt(1 / (2 * R * ebno[0])))], name="transmit1")

signal_input2 = Sequential(
    [Lambda(TransmissionLayer, arguments={"t":0, "k":1}),
    GaussianNoise(np.sqrt(1 / (2 * R * ebno[1])))], name="transmit2")

decoder1 = Sequential([
    Dense(M, activation="relu"),
    Dense(M, activation="relu"),
    Dense(M, activation="softmax")
], name="decoder1")

decoder2 = Sequential([
    Dense(M, activation="relu"),
    Dense(M, activation="relu"),
    Dense(M, activation="softmax")
], name="decoder2")

In [6]:
x1 = encoder1(input_signal1)
x2 = encoder2(input_signal2)
x = concatenate([x1, x2], axis=1)
x = combiner(x)
x1 = signal_input1(x)
x2 = signal_input2(x)
out1 = decoder1(x1)
out2 = decoder2(x2)

model = Model(inputs=[input_signal1, input_signal2], outputs=[out1, out2])
model.summary()
alpha = K.variable(.5)

model.compile(optimizer="adam", loss=["categorical_crossentropy", "categorical_crossentropy"],
            loss_weights=[alpha, (1 - alpha)], metrics=["accuracy"])




Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             (None, 4)            0                                            
__________________________________________________________________________________________________
input2 (InputLayer)             (None, 4)            0                                            
__________________________________________________________________________________________________
encoder1 (Sequential)           (None, 2)            30          input1[0][0]                     
__________________________________________________________________________________________________
encoder2 (Sequential)           (None, 2)            30          input2[0][0]                     
_________________________________________________________________________________________

In [ ]:
train_datas = generate_train_datas()

model.fit(x=train_datas, y=train_datas, validation_split=.5, batch_size=10000,
    epochs=2000, callbacks=[
    EarlyStopping(patience=100, restore_best_weights=True, monitor="loss"),
    ReduceLROnPlateau(monitor="loss", factor=.5, patience=20),
    AlphaCallback(alpha)
])

Train on 200000 samples, validate on 200000 samples
Epoch 1/2000
200000/200000 [==============================] - 0s 2us/step - loss: 1.0439 - decoder1_loss: 1.0249 - decoder2_loss: 1.0622 - decoder1_acc: 0.4834 - decoder2_acc: 0.4473 - val_loss: 1.1115 - val_decoder1_loss: 1.1281 - val_decoder2_loss: 1.0953 - val_decoder1_acc: 0.3757 - val_decoder2_acc: 0.4373
Epoch 2/2000
200000/200000 [==============================] - 0s 2us/step - loss: 1.0196 - decoder1_loss: 1.0009 - decoder2_loss: 1.0377 - decoder1_acc: 0.4854 - decoder2_acc: 0.5475 - val_loss: 1.0824 - val_decoder1_loss: 1.1025 - val_decoder2_loss: 1.0633 - val_decoder1_acc: 0.3757 - val_decoder2_acc: 0.6252
Epoch 3/2000
200000/200000 [==============================] - 0s 2us/step - loss: 0.9982 - decoder1_loss: 0.9815 - decoder2_loss: 1.0143 - decoder1_acc: 0.4897 - decoder2_acc: 0.5687 - val_loss: 1.0444 - val_decoder1_loss: 1.0602 - val_decoder2_loss: 1.0292 - val_decoder1_acc: 0.3757 - val_decoder2_acc: 0.6252
Epoch 4/2000

200000/200000 [==============================] - 0s 2us/step - loss: 0.6962 - decoder1_loss: 0.5552 - decoder2_loss: 0.7947 - decoder1_acc: 0.7003 - decoder2_acc: 0.6052 - val_loss: 0.7512 - val_decoder1_loss: 0.6615 - val_decoder2_loss: 0.8132 - val_decoder1_acc: 0.6244 - val_decoder2_acc: 0.6252
Epoch 28/2000
200000/200000 [==============================] - 0s 2us/step - loss: 0.6892 - decoder1_loss: 0.5473 - decoder2_loss: 0.7878 - decoder1_acc: 0.7015 - decoder2_acc: 0.6065 - val_loss: 0.7195 - val_decoder1_loss: 0.6157 - val_decoder2_loss: 0.7917 - val_decoder1_acc: 0.6244 - val_decoder2_acc: 0.6252
Epoch 29/2000
200000/200000 [==============================] - 0s 2us/step - loss: 0.6825 - decoder1_loss: 0.5384 - decoder2_loss: 0.7817 - decoder1_acc: 0.7042 - decoder2_acc: 0.6071 - val_loss: 0.7134 - val_decoder1_loss: 0.6182 - val_decoder2_loss: 0.7798 - val_decoder1_acc: 0.5621 - val_decoder2_acc: 0.6252
Epoch 30/2000
200000/200000 [==============================] - 0s 2us/step 